In [2]:
# Import library to open dataset
import opendatasets as od
# Import python libraries
import pandas as pd
import numpy as np

# Import libraries for visualization 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from wordcloud import WordCloud
import folium

# Folium Map 

In [3]:
divvy_master = pd.read_csv('/Users/deepasarojam/Documents/Kevin Hatman - Data Viz/Data/divvy_station_master_data_201909-202108.csv')

In [4]:
divvy_master.head()

,old_station_id,new_station_id,station_name,total_docks,total_docs_in_service,status,latitude,longitude,distance,address,...,street,zip_code,chicago_neighborhood_name,chicago_neighborhood_number,chicago_community_area_name,chicago_community_area_number,city,county,total_starts,total_destinations
0,456,456,2112 W Peterson Ave,11,11,In Service,41.991178,-87.683593,19.796532,"2112 W Peterson Ave, Chicago, IL 60659, United...",...,W Peterson Ave,60659,West Rogers Park,2.5,West Ridge,2.0,Chicago,Cook County,1754.0,1949.0
1,101,15491,63rd St Beach,15,15,In Service,41.780911,-87.576324,38.239093,"1952 East Hayes Drive, Chicago, IL 60637, Unit...",...,East Hayes Drive,60637,Woodlawn,42,Woodlawn,42.0,Chicago,Cook County,4493.0,4574.0
2,109,13028,900 W Harrison St,19,19,In Service,41.874754,-87.649807,649.423794,"University of Illinois at Chicago, 1200 West H...",...,West Harrison Street,60607,Near West Side,28,Near West Side,28.0,Chicago,Cook County,11940.0,11749.0
3,21,13157,Aberdeen St & Jackson Blvd,23,23,In Service,41.877726,-87.654787,0.004159,"Aberdeen St & Jackson Blvd, West Jackson Boule...",...,West Jackson Boulevard,60607,Near West Side,28,Near West Side,28.0,Chicago,Cook County,20573.0,20425.0
4,80,13156,Aberdeen St & Monroe St,19,19,In Service,41.880419,-87.655519,20.917140,"1100 West Monroe Street, Chicago, IL 60607, Un...",...,West Monroe Street,60607,Near West Side,28,Near West Side,28.0,Chicago,Cook County,17316.0,17517.0


In [5]:
divvy_master.columns

Index(['old_station_id', 'new_station_id', 'station_name', 'total_docks',
       'total_docs_in_service', 'status', 'latitude', 'longitude', 'distance',
       'address', 'nearest_house_number', 'corner_name', 'street', 'zip_code',
       'chicago_neighborhood_name', 'chicago_neighborhood_number',
       'chicago_community_area_name', 'chicago_community_area_number', 'city',
       'county', 'total_starts', 'total_destinations'],
      dtype='object')

The first thing to do is to create a map object centered around the area of interest. I also set them zoom level to 12 in order to see the individual points. I also create a popup tooltip to provide the name of each counter. The default map will be displyed as `OpenStreetMap tiles`. 

In [6]:
import folium

chicago_m = folium.Map(location=[41.881832, -87.623177], zoom_start=12)


for lat, lng, station in zip(divvy_master['latitude'],
                              divvy_master['longitude'],
                              divvy_master['station_name']):
                              
                              folium.Marker([lat, lng], popup=[station, lat, lng]
                              ).add_to(chicago_m)

chicago_m

I displayed the station along with the latitude and longitude of a few random individual records to make sure they are plotted correctly. I also decided to change the look of the point icons. I changed their color to dark blue. I also added a bicycle icon to the markers in color `white`. 

In [7]:
import folium

chicago_m = folium.Map(location=[41.881832, -87.623177], zoom_start=12)


for lat, lng, station in zip(divvy_master['latitude'],
                              divvy_master['longitude'],
                              divvy_master['station_name']):
                              
                              folium.Marker([lat, lng], popup=station, icon=folium.Icon(color='darkblue', icon_color='white', icon='bicycle', angle=0, prefix='fa')
                              ).add_to(chicago_m)

chicago_m

The above map is not too bad, but the map is too busy. Many data points are not visible when the map is zoomed to zoom=12. Several points are hard to see since they can be very close to each other. To fix this issue, I reduced the zoom level to 11, and created marker clusters. This will also color mark the busiest stations as red, orange, green.

In [8]:
import folium
from folium.plugins import MarkerCluster
MarkerCluster()

chicago_m2 = folium.Map(location=[41.881832, -87.623177], zoom_start=11)

marker_cluster = MarkerCluster().add_to(chicago_m2)

for lat, lng, station, starts, dest in zip(divvy_master['latitude'],
                              divvy_master['longitude'],
                              divvy_master['station_name'],
                              divvy_master['total_starts'],
                              divvy_master['total_destinations']):
                              
                              label = 'Station: {}'.format(station) #+  'Total Starts: {}'.format(starts) + 'Total Destinations: {}'.format(dest)

                              folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='darkblue', icon_color='white', icon='bicycle', angle=0, prefix='fa')
                            ).add_to(marker_cluster)
chicago_m2